In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import cv2
import os
import torch
import torchvision
import torchinfo
import skimage

from torch.utils.data import Dataset, DataLoader

EPOCHS = 100
BATCH_SIZE = 256
LR = 2e-4

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(f"Device: {device}")

Device: cuda


In [5]:
images = torchvision.datasets.MNIST("datasets", download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: datasets
    Split: Train
